In [2]:
import pandas as pd
import numpy as np

In [39]:
df_none = pd.read_csv('/home/jupyter/datasphere/project/df_none.csv')
df_none.head()

,is_valid,od_color,od_monotone,od_size,od_shape,od_border,od_excavation_size,od_excavation_location,od_excavation_ratio,od_vessels_location,od_pathology,vessels_art_course,vessels_art_turtuosity,vessels_art_bifurcation,vessels_art_caliber,vessels_vein_course,vessels_vein_turtuosity,vessels_vein_bifurcation,vessels_vein_caliber,vessels_ratio,vessels_pathology,macula_macular_reflex,macula_foveal_reflex,macula_pathology,peripheral_pathology,other_diagnosis,other_notes,text
0,True,бледно-розовый,NaN,NaN,NaN,четкие,NaN,NaN,NaN,NaN,NaN,нормальное,нормальная,NaN,нормальный,NaN,NaN,NaN,нормальный,NaN,NaN,четкий,NaN,слегка гиперпигментирована,участки разряжения пигмента,альбинотическое глазное дно,NaN,"ДЗН бледно-розовый, границы четкие, ход и кали..."
1,True,/Events,/Events,/Events,/Events,размытые,NaN,NaN,NaN,NaN,/Events,растянутые,расширенные,/Events,NaN,norman,expanded,none,NaN,NaN,MistVenin Varie rise>: отличистрерство,ums,/Refle,NaN,Normative!,Donfuraxe,Sons,"За флером: границы ДЗН нечеткие, сосуды расшир..."
2,True,бледно-розовый,NaN,NaN,NaN,четкие,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,не изменен,NaN,NaN,NaN,не изменен,NaN,NaN,ровный,NaN,NaN,без патологии,NaN,NaN,"ДЗН бледно-розовый, с четкими границами, ход и..."
3,True,бледно-розовый,NaN,NaN,NaN,четкие,NaN,NaN,NaN,в височную сторону,NaN,NaN,несколько извиты,NaN,сужены,NaN,несколько извиты,NaN,NaN,NaN,NaN,сглажены,NaN,NaN,"аваскулярная зона, локальная отслойка сетчатки...",NaN,NaN,"ДЗН бледно-розовый, границы четкие, артерии не..."
4,True,бледный,NaN,гипопластичен,NaN,NaN,NaN,NaN,NaN,NaN,Диск зрительного нерва деформирован тракционно...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Описывается аваскулярная сетчатка с перераспре...,NaN,NaN,"ДЗН бледный, гипопластичен, тракционно деформи..."


In [40]:
df_none = df_none[df_none['is_valid']==True]
df_none = df_none.drop(columns=['is_valid', 'text'])

In [41]:
np.mean(df_none.count(axis=1))

7.530183727034121

In [42]:
df_none.shape

(762, 26)

In [43]:
np.round((df_none.count() / df_none.shape[0]).sort_values(ascending=False), 2)

od_color                    0.95
od_border                   0.90
peripheral_pathology        0.78
vessels_art_caliber         0.57
vessels_vein_caliber        0.45
macula_macular_reflex       0.44
macula_pathology            0.41
vessels_art_turtuosity      0.37
vessels_art_course          0.35
vessels_vein_turtuosity     0.35
vessels_vein_course         0.31
od_pathology                0.30
od_monotone                 0.22
macula_foveal_reflex        0.19
od_vessels_location         0.15
vessels_pathology           0.12
other_notes                 0.11
vessels_art_bifurcation     0.11
vessels_vein_bifurcation    0.10
od_size                     0.07
od_shape                    0.05
other_diagnosis             0.05
od_excavation_size          0.05
od_excavation_location      0.04
vessels_ratio               0.03
od_excavation_ratio         0.03
dtype: float64

In [44]:
df_none_x_potology = df_none.drop(columns=['vessels_pathology', 'od_pathology', 'macula_pathology', 'peripheral_pathology', 'other_diagnosis', 'other_notes'],)

In [45]:
np.mean(df_none_x_potology.count(axis=1))

5.745406824146982

In [47]:
np.round((df_none_x_potology.count() / df_none.shape[0]).sort_values(ascending=False), 2)

od_color                    0.95
od_border                   0.90
vessels_art_caliber         0.57
vessels_vein_caliber        0.45
macula_macular_reflex       0.44
vessels_art_turtuosity      0.37
vessels_art_course          0.35
vessels_vein_turtuosity     0.35
vessels_vein_course         0.31
od_monotone                 0.22
macula_foveal_reflex        0.19
od_vessels_location         0.15
vessels_art_bifurcation     0.11
vessels_vein_bifurcation    0.10
od_size                     0.07
od_shape                    0.05
od_excavation_size          0.05
od_excavation_location      0.04
vessels_ratio               0.03
od_excavation_ratio         0.03
dtype: float64

In [30]:
# for column in df_none_x_potology.columns:
#     display(df_none_x_potology[column].value_counts())
#     print('='*150)

In [3]:
df_none2 = pd.read_csv('/home/jupyter/datasphere/project/df_none2.csv')
df_none2.head()

,is_valid,od_color,od_monotone,od_size,od_shape,od_border,od_excavation_size,od_excavation_location,od_excavation_ratio,od_vessels_location,od_pathology,vessels_art_course,vessels_art_turtuosity,vessels_art_bifurcation,vessels_art_caliber,vessels_vein_course,vessels_vein_turtuosity,vessels_vein_bifurcation,vessels_vein_caliber,vessels_ratio,vessels_pathology,macula_macular_reflex,macula_foveal_reflex,macula_pathology,peripheral_pathology,other_diagnosis,other_notes,text
0,True,бледно-розовый,NaN,NaN,NaN,четкие,NaN,NaN,NaN,NaN,NaN,нормальное,нормальная,NaN,нормальный,NaN,NaN,NaN,нормальный,NaN,NaN,четкий,NaN,слегка гиперпигментирована,участки разряжения пигмента,альбинотическое глазное дно,NaN,"ДЗН бледно-розовый, границы четкие, ход и кали..."
1,True,/Events,/Events,/Events,/Events,размытые,NaN,NaN,NaN,NaN,/Events,растянутые,расширенные,/Events,NaN,norman,expanded,none,NaN,NaN,MistVenin Varie rise>: отличистрерство,ums,/Refle,NaN,Normative!,Donfuraxe,Sons,"За флером: границы ДЗН нечеткие, сосуды расшир..."
2,True,бледно-розовый,NaN,NaN,NaN,четкие,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,не изменен,NaN,NaN,NaN,не изменен,NaN,NaN,ровный,NaN,NaN,без патологии,NaN,NaN,"ДЗН бледно-розовый, с четкими границами, ход и..."
3,True,бледно-розовый,NaN,NaN,NaN,четкие,NaN,NaN,NaN,в височную сторону,NaN,NaN,несколько извиты,NaN,сужены,NaN,несколько извиты,NaN,NaN,NaN,NaN,сглажены,NaN,NaN,"аваскулярная зона, локальная отслойка сетчатки...",NaN,NaN,"ДЗН бледно-розовый, границы четкие, артерии не..."
4,True,бледный,NaN,гипопластичен,NaN,NaN,NaN,NaN,NaN,NaN,Диск зрительного нерва деформирован тракционно...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Описывается аваскулярная сетчатка с перераспре...,NaN,NaN,"ДЗН бледный, гипопластичен, тракционно деформи..."


Перевод датафрейма в набор JSON-ов

In [4]:
import pandas as pd
import json

def transform_row_to_json(row):
    return ({
        "ДЗН": {
            "Цвет": row["od_color"],
            "Монотонность": bool(row["od_monotone"]),
            "Размер": row["od_size"],
            "Форма": row["od_shape"],
            "Границы": row["od_border"],
            "Экскавация": {
                "Размер": row["od_excavation_size"],
                "Сектор": row["od_excavation_location"]
            },
            "Э/Д": row["od_excavation_ratio"],
            "Сосудистый пучок": row["od_vessels_location"],
            "Патология": row["od_pathology"] or ""
        },
        "Сосуды": {
            "Артерии": {
                "Ход": row["vessels_art_course"],
                "Извитость": row["vessels_art_turtuosity"],
                "Бифуркация": row["vessels_art_bifurcation"],
                "Калибр": row["vessels_art_caliber"]
            },
            "Вены": {
                "Ход": row["vessels_vein_course"],
                "Извитость": row["vessels_vein_turtuosity"],
                "Бифуркация": row["vessels_vein_bifurcation"],
                "Калибр": row["vessels_vein_caliber"]
            },
            "А/В индекс": row["vessels_ratio"],
            "Патология": row["vessels_pathology"] or ""
        },
        "Макула": {
            "Макулярный рефлекс": row["macula_macular_reflex"],
            "Фовеальный рефлекс": row["macula_foveal_reflex"],
            "Патология": row["macula_pathology"] or ""
        },
        "Периферия": {
            "Патология": row["peripheral_pathology"] or ""
        },
        "Другое": {
            "Диагноз": row["other_diagnosis"] or "",
            "Пометки": row["other_notes"] or ""
        }
    }, row['text'])

def dataframe_to_json(df):
    return df.apply(transform_row_to_json, axis=1).tolist()

In [5]:
json_array = dataframe_to_json(df_none2)

In [6]:
json_array[7][1]

'ДЗН-бледно-розовый,проминирует, границы размыты, ход и калибр сосудов не нарушен, макула по возрасту, очаговых симптомов нет'

In [17]:
json_array[14]

({'ДЗН': {'Цвет': 'бледно-розовый',
   'Монотонность': True,
   'Размер': nan,
   'Форма': nan,
   'Границы': 'четкие',
   'Экскавация': {'Размер': nan, 'Сектор': nan},
   'Э/Д': nan,
   'Сосудистый пучок': nan,
   'Патология': nan},
  'Сосуды': {'Артерии': {'Ход': nan,
    'Извитость': nan,
    'Бифуркация': nan,
    'Калибр': 'нормальный'},
   'Вены': {'Ход': nan,
    'Извитость': 'Nonde',
    'Бифуркация': nan,
    'Калибр': 'нормальный'},
   'А/В индекс': nan,
   'Патология': nan},
  'Макула': {'Макулярный рефлекс': nan,
   'Фовеальный рефлекс': nan,
   'Патология': nan},
  'Периферия': {'Патология': 'без особенностей'},
  'Другое': {'Диагноз': nan, 'Пометки': nan}},
 'ДЗН - бледно-розовый, границы четкие, калибр сосудов не изменен, перефирия без особенностей')

In [18]:
df_fundus_descr = pd.read_csv('/home/jupyter/datasphere/project/fundus_descriptions_1020.csv')

In [24]:
df_fundus_descr[df_fundus_descr['text']=='ДЗН - бледно-розовый, границы четкие, калибр сосудов не изменен, перефирия без особенностей']

,is_valid,od_color,od_monotone,od_size,od_shape,od_border,od_excavation_size,od_excavation_location,od_excavation_ratio,od_vessels_location,od_pathology,vessels_art_course,vessels_art_turtuosity,vessels_art_bifurcation,vessels_art_caliber,vessels_vein_course,vessels_vein_turtuosity,vessels_vein_bifurcation,vessels_vein_caliber,vessels_ratio,vessels_pathology,macula_macular_reflex,macula_foveal_reflex,macula_pathology,peripheral_pathology,other_diagnosis,other_notes,text
17,True,бледно-розовый,True,нормальный,правильная,четкие,нормальный,в центре,NaN,в центре,NaN,нормальный,нормальная,нормальная,нормальный,нормальный,нормальная,нормальная,нормальный,NaN,NaN,нормальный,нормальный,NaN,без особенностей,NaN,NaN,"ДЗН - бледно-розовый, границы четкие, калибр с..."


In [21]:
fundus_json = dataframe_to_json(df_fundus_descr) 

In [25]:
fundus_json[17]

({'ДЗН': {'Цвет': 'бледно-розовый',
   'Монотонность': True,
   'Размер': 'нормальный',
   'Форма': 'правильная',
   'Границы': 'четкие',
   'Экскавация': {'Размер': 'нормальный', 'Сектор': 'в центре'},
   'Э/Д': nan,
   'Сосудистый пучок': 'в центре',
   'Патология': nan},
  'Сосуды': {'Артерии': {'Ход': 'нормальный',
    'Извитость': 'нормальная',
    'Бифуркация': 'нормальная',
    'Калибр': 'нормальный'},
   'Вены': {'Ход': 'нормальный',
    'Извитость': 'нормальная',
    'Бифуркация': 'нормальная',
    'Калибр': 'нормальный'},
   'А/В индекс': nan,
   'Патология': nan},
  'Макула': {'Макулярный рефлекс': 'нормальный',
   'Фовеальный рефлекс': 'нормальный',
   'Патология': nan},
  'Периферия': {'Патология': 'без особенностей'},
  'Другое': {'Диагноз': nan, 'Пометки': nan}},
 'ДЗН - бледно-розовый, границы четкие, калибр сосудов не изменен, перефирия без особенностей')